<a href="https://colab.research.google.com/github/ParissaAmin/Anomaly-Detection-in-Time-Series/blob/main/2nd%20Anomaly_Detection_in_Transactions_using_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the necessary Python libraries

In [33]:
import pandas as pd
import plotly.express as px

from sklearn.model_selection import train_test_split
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report

In [19]:
from google.colab import files
uploaded = files.upload()

Saving transaction_anomalies_dataset.csv to transaction_anomalies_dataset (5).csv


In [20]:
#Saving files in colab
from google.colab import drive
# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
!cat transaction_anomalies_dataset.csv

Transaction_ID,Transaction_Amount,Transaction_Volume,Average_Transaction_Amount,Frequency_of_Transactions,Time_Since_Last_Transaction,Day_of_Week,Time_of_Day,Age,Gender,Income,Account_Type
TX0,1024.8357076505617,3,997.2347139765764,12,29,Friday,06:00,36,Male,1436074,Savings
TX1,1013.9520646100069,4,1020.2103056961305,7,22,Friday,01:00,41,Female,627069,Savings
TX2,970.9560932988243,1,989.496603856437,5,12,Tuesday,21:00,61,Male,786232,Savings
TX3,1040.8222540475663,2,969.5224800476827,16,28,Sunday,14:00,61,Male,619030,Savings
TX4,998.7772411252968,1,1007.1110263424487,7,7,Friday,08:00,56,Female,649457,Savings
TX5,969.9873074947044,1,1018.9487964209328,6,20,Thursday,14:00,24,Male,872843,Savings
TX6,973.3175598078757,2,999.8894427535975,5,4,Tuesday,07:00,21,Male,1105889,Savings
TX7,931.1030348088759,4,992.9376669598587,10,29,Sunday,17:00,43,Male,502283,Savings
TX8,1036.9233289997705,4,1003.4273656237995,18,23,Thursday,07:00,38,Female,323247,Savings
TX9,1013.1743020115105,4,1032.83542232028

In [27]:
!head transaction_anomalies_dataset.csv

Transaction_ID,Transaction_Amount,Transaction_Volume,Average_Transaction_Amount,Frequency_of_Transactions,Time_Since_Last_Transaction,Day_of_Week,Time_of_Day,Age,Gender,Income,Account_Type
TX0,1024.8357076505617,3,997.2347139765764,12,29,Friday,06:00,36,Male,1436074,Savings
TX1,1013.9520646100069,4,1020.2103056961305,7,22,Friday,01:00,41,Female,627069,Savings
TX2,970.9560932988243,1,989.496603856437,5,12,Tuesday,21:00,61,Male,786232,Savings
TX3,1040.8222540475663,2,969.5224800476827,16,28,Sunday,14:00,61,Male,619030,Savings
TX4,998.7772411252968,1,1007.1110263424487,7,7,Friday,08:00,56,Female,649457,Savings
TX5,969.9873074947044,1,1018.9487964209328,6,20,Thursday,14:00,24,Male,872843,Savings
TX6,973.3175598078757,2,999.8894427535975,5,4,Tuesday,07:00,21,Male,1105889,Savings
TX7,931.1030348088759,4,992.9376669598587,10,29,Sunday,17:00,43,Male,502283,Savings
TX8,1036.9233289997705,4,1003.4273656237995,18,23,Thursday,07:00,38,Female,323247,Savings


In [25]:
# Loading data from CSV file
for transaction_anomalies_dataset in uploaded.keys():
    data = pd.read_csv(transaction_anomalies_dataset)

In [26]:
#If the data has any null values
print(data.isnull().sum())

Transaction_ID                 0
Transaction_Amount             0
Transaction_Volume             0
Average_Transaction_Amount     0
Frequency_of_Transactions      0
Time_Since_Last_Transaction    0
Day_of_Week                    0
Time_of_Day                    0
Age                            0
Gender                         0
Income                         0
Account_Type                   0
dtype: int64


In [28]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Transaction_ID               1000 non-null   object 
 1   Transaction_Amount           1000 non-null   float64
 2   Transaction_Volume           1000 non-null   int64  
 3   Average_Transaction_Amount   1000 non-null   float64
 4   Frequency_of_Transactions    1000 non-null   int64  
 5   Time_Since_Last_Transaction  1000 non-null   int64  
 6   Day_of_Week                  1000 non-null   object 
 7   Time_of_Day                  1000 non-null   object 
 8   Age                          1000 non-null   int64  
 9   Gender                       1000 non-null   object 
 10  Income                       1000 non-null   int64  
 11  Account_Type                 1000 non-null   object 
dtypes: float64(2), int64(5), object(5)
memory usage: 93.9+ KB
None


In [29]:
print(data.describe())

       Transaction_Amount  Transaction_Volume  Average_Transaction_Amount  \
count         1000.000000         1000.000000                 1000.000000   
mean          1038.122511            2.498000                 1000.682506   
std            283.580055            1.115006                   20.632334   
min            849.024392            1.000000                  939.081423   
25%            966.028796            1.000000                  986.800556   
50%           1002.118678            3.000000                 1000.501902   
75%           1033.143657            3.000000                 1015.155595   
max           3227.459108            4.000000                 1073.154036   

       Frequency_of_Transactions  Time_Since_Last_Transaction          Age  \
count                1000.000000                  1000.000000  1000.000000   
mean                   12.078000                    15.341000    40.641000   
std                     4.245225                     8.361258    13.8199

# looking at the distribution of transactions amount in the data

In [41]:

# Distribution of Transaction Amount

fig_amount = px.histogram(data, x='Transaction_Amount',
                          nbins=20,
                          title='Distribution of Transaction Amount')


fig_amount.update_layout(height=600, width=800)  # Adjusting height and width

fig_amount.show()

#looking at the distribution of transactions amount by account type

In [44]:
# Transaction Amount by Account Type

fig_box_amount = px.box(data,
                        x='Account_Type',
                        y='Transaction_Amount',
                        title='Transaction Amount by Account Type')

fig_box_amount.update_layout(height=600, width=800)  # Adjusting height and width

fig_box_amount.show()

In [45]:
# Average Transaction Amount vs. Age
fig_scatter_avg_amount_age = px.scatter(data, x='Age',
                                        y='Average_Transaction_Amount',
                                        color='Account_Type',
                                        title='Average Transaction Amount vs. Age',
                                        trendline='ols')

fig_scatter_avg_amount_age .update_layout(height=600, width=800)

fig_scatter_avg_amount_age.show()

There’s no difference in the average transaction amount by age

#The count of transactions by day of the week

In [46]:
fig_day_of_week = px.bar(data, x='Day_of_Week',
                         title='Count of Transactions by Day of the Week')

fig_day_of_week.update_layout(height=600, width=800)

fig_day_of_week.show()

# The correlation between all the columns in the data

In [47]:
# Correlation Heatmap
correlation_matrix = data.corr()
fig_corr_heatmap = px.imshow(correlation_matrix,
                             title='Correlation Heatmap')

fig_corr_heatmap.update_layout(height=600, width=800)

fig_corr_heatmap.show()

<ipython-input-47-ad7006b5b7af>:2: FutureWarning:

The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.



#Visualizing anomalies in the data

In [49]:
# Calculating mean and standard deviation of Transaction Amount

mean_amount = data['Transaction_Amount'].mean()
std_amount = data['Transaction_Amount'].std()


# Defining the anomaly threshold (2 standard deviations from the mean)
anomaly_threshold = mean_amount + 2 * std_amount


#Flaging anomalies
data['Is_Anomaly'] = data['Transaction_Amount'] > anomaly_threshold


# Scatter plot of Transaction Amount with anomalies highlighted
fig_anomalies = px.scatter(data, x='Transaction_Amount', y='Average_Transaction_Amount',
                           color='Is_Anomaly', title='Anomalies in Transaction Amount')
fig_anomalies.update_traces(marker=dict(size=12),
                            selector=dict(mode='markers', marker_size=1))

fig_anomalies.update_layout(height=600, width=800)

fig_anomalies.show()

#calculating the number of anomalies

In [50]:
num_anomalies = data['Is_Anomaly'].sum()

# Calculating the total number of instances in the dataset
total_instances = data.shape[0]


# Calculating the ratio of anomalies
anomaly_ratio = num_anomalies / total_instances

print(anomaly_ratio)

0.02


#Training a Machine Learning model for detecting anomalies

we are training an anomaly detection model using the Isolation Forest algorithm

In [51]:
relevant_features = ['Transaction_Amount',
                     'Average_Transaction_Amount',
                     'Frequency_of_Transactions']

# Spliting data into features (X) and target variable (y)

X = data[relevant_features]
y = data['Is_Anomaly']

# Spliting data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training the Isolation Forest model

model = IsolationForest(contamination=0.02, random_state=42)

model.fit(X_train)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning:

X does not have valid feature names, but IsolationForest was fitted with feature names



IsolationForest(contamination=0.02, random_state=42)

#Predicting anomalies on the test set

In [52]:
y_pred = model.predict(X_test)

# Convert predictions to binary values (0: normal, 1: anomaly)

y_pred_binary = [1 if pred == -1 else 0 for pred in y_pred]


# Evaluate the model's performance

report = classification_report(y_test, y_pred_binary, target_names=['Normal', 'Anomaly'])
print(report)

              precision    recall  f1-score   support

      Normal       1.00      1.00      1.00       196
     Anomaly       1.00      1.00      1.00         4

    accuracy                           1.00       200
   macro avg       1.00      1.00      1.00       200
weighted avg       1.00      1.00      1.00       200



#Using trained model to detect anomalies

In [53]:
#features used during training
relevant_features = ['Transaction_Amount', 'Average_Transaction_Amount', 'Frequency_of_Transactions']


# Getinguser inputs for features
user_inputs = []
for feature in relevant_features:
    user_input = float(input(f"Enter the value for '{feature}': "))
    user_inputs.append(user_input)


# Creating a DataFrame from user inputs
user_df = pd.DataFrame([user_inputs], columns=relevant_features)


# Predict anomalies using the model
user_anomaly_pred = model.predict(user_df)


# Converting the prediction to binary value (0: normal, 1: anomaly)
user_anomaly_pred_binary = 1 if user_anomaly_pred == -1 else 0


if user_anomaly_pred_binary == 1:
    print("Anomaly detected: This transaction is flagged as an anomaly.")
else:
    print("No anomaly detected: This transaction is normal.")

Enter the value for 'Transaction_Amount': 10000
Enter the value for 'Average_Transaction_Amount':  900
Enter the value for 'Frequency_of_Transactions': 6
Anomaly detected: This transaction is flagged as an anomaly.
